# Imports

In [81]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import RFECV, RFE
import xgboost

from skopt import BayesSearchCV
from skopt.plots import plot_objective
from skopt.space import Real, Categorical, Integer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MaxAbsScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import PredefinedSplit


import optuna

import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb
from helpers import *

from sklearn.model_selection import cross_val_score

# auto reloading library (mainly for altering helpers.py)
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
def merge_largeX_smallY(X,y,merge_type='mean'):
    # Add time column that only holds the hour
    X['time'] = X['date_forecast'].dt.floor('H')

    
    if merge_type == 'mean':
        # Perform Transformation from 4 to 1 values per hour
        X = X.groupby(['building_id', 'time']).mean().reset_index()
        y = y
    else:
        raise ValueError(f'merge_type \'{merge_type}\' not supported')
    
    # Merge X and y
    Xy = pd.merge(X, y, on=['building_id', 'time']).reset_index(drop=True)
    # Drop the time column
    Xy = Xy.drop(columns=['time'])
    return Xy

In [83]:
def get_data():
    # Read data
    train_a = pd.read_parquet('A/train_targets.parquet')
    train_b = pd.read_parquet('B/train_targets.parquet')
    train_c = pd.read_parquet('C/train_targets.parquet')

    X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
    X_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
    X_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')

    X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
    X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
    X_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')

    X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
    X_test_estimated_b = pd.read_parquet('B/X_test_estimated.parquet')
    X_test_estimated_c = pd.read_parquet('C/X_test_estimated.parquet')

    # Add building_id
    train_a['building_id'] = 'a'
    train_b['building_id'] = 'b'
    train_c['building_id'] = 'c'

    X_train_estimated_a['building_id'] = 'a'
    X_train_estimated_b['building_id'] = 'b'
    X_train_estimated_c['building_id'] = 'c'

    X_train_observed_a['building_id'] = 'a'
    X_train_observed_b['building_id'] = 'b'
    X_train_observed_c['building_id'] = 'c'

    X_test_estimated_a['building_id'] = 'a'
    X_test_estimated_b['building_id'] = 'b'
    X_test_estimated_c['building_id'] = 'c'

    # Combine Data
    X_o = pd.concat([X_train_observed_a, X_train_observed_b, X_train_observed_c])
    X_e = pd.concat([X_train_estimated_a, X_train_estimated_b, X_train_estimated_c])
    X_submission = pd.concat([X_test_estimated_a, X_test_estimated_b, X_test_estimated_c])
    y = pd.concat([train_a, train_b, train_c])

    # Add isEstimated column
    X_o['isEstimated'] = 0
    X_e['isEstimated'] = 1
    X_submission['isEstimated'] = 1


    # Combine
    X = pd.concat([X_o, X_e])

    return X, y, X_submission

In [84]:
def get_splitted_data(merge_type='mean', split_strategy='2021_summer'):
    X, y, X_submission = get_data()
    # Merge X and y
    Xy = merge_largeX_smallY(X,y, merge_type=merge_type)

    # Split into train and test
    if split_strategy == '2021_summer':
        val_idx = Xy.date_forecast.between('2021-05-01', '2021-07-31')
    else:
        raise ValueError(f'split_strategy \'{split_strategy}\' not supported')

    Xy_train = Xy[~val_idx]
    Xy_val = Xy[val_idx]


    # Split into X and y
    X_train = Xy_train.drop(['pv_measurement'], axis=1)
    y_train = Xy_train['pv_measurement']

    X_val = Xy_val.drop(['pv_measurement'], axis=1)
    y_val = Xy_val['pv_measurement']

    return X_train, y_train, X_val, y_val

In [85]:
def add_features_time(X):
    # Add monthYear column
    X['monthYear'] = X['date_forecast'].dt.to_period('M')

    # Add dayMonthYear column
    X['dayMonthYear'] = X['date_forecast'].dt.to_period('D')

    # Add month column
    X['month'] = X['date_forecast'].dt.month

    # Add week column
    #X['week'] = X['date_forecast'].dt.week

    # Add day column
    X['day'] = X['date_forecast'].dt.day

    # Add hour column
    X['hour'] = X['date_forecast'].dt.hour

    # Calculate difference between date_forecast and date_calc
    X['delta_forecast'] = (X['date_forecast']-X['date_calc']).apply(lambda x: x.total_seconds() / 3600)

    return X

In [86]:
def add_features_general(X, norm_radiation_cap=None):
    max_val = 1

    X['GHI'] = X['diffuse_rad:W'] + X['direct_rad:W']
    X['wind_angle'] = np.arctan2(X['wind_speed_u_10m:ms'], X['wind_speed_v_10m:ms'])
    X['norm_radiation'] = (X['GHI'] / X['clear_sky_rad:W']).fillna(0)
    if norm_radiation_cap:
        X.loc[X['norm_radiation'] > norm_radiation_cap, 'norm_radiation'] = norm_radiation_cap
    
    return X


In [87]:
def add_features_lag(X, na_fill_value=None):
    # lagged feature of GHI
    X['GHI_lag-2'] = X.groupby('building_id')['GHI'].shift(-2)
    X['GHI_lag-1'] = X.groupby('building_id')['GHI'].shift(-1)
    X['GHI_lag1'] = X.groupby('building_id')['GHI'].shift(1)
    X['GHI_lag2'] = X.groupby('building_id')['GHI'].shift(2)

    if na_fill_value is None:
        X = X.dropna(subset=['GHI_lag-2', 'GHI_lag-1', 'GHI_lag1', 'GHI_lag2'])
    else:
        print('WADDEHADDEDUDEDA')
        print(na_fill_value)
        X = X.fillna(na_fill_value)


    return X


In [88]:
def add_features_interaction(X):
    # general, somewhat intuitive interactions
    X['temp*GHI'] = X['GHI'] * X['t_1000hPa:K']
    X['wind*humidity'] = X['wind_speed_10m:ms'] * X['relative_humidity_1000hPa:p']
    X['sun_height*diff_rad'] = X['sun_elevation:d'] * X['diffuse_rad:W']

    # chat GPT
    X['hour*wind_speed_10m'] = X['hour'] * X['wind_speed_10m:ms']
    X['hour*clear_sky_rad'] = X['hour'] * X['clear_sky_rad:W']
    X['month*sun_elevation'] = X['month'] * X['sun_elevation:d']

    X['relative_humidity*air_density'] = X['relative_humidity_1000hPa:p'] * X['air_density_2m:kgm3']
    X['temperature*wind_speed'] = X['t_1000hPa:K'] * X['wind_speed_10m:ms']

    X['GHI*clear_sky_energy'] = X['GHI'] * X['clear_sky_energy_1h:J']
    X['GHI*sun_azimuth'] = X['GHI'] * X['sun_azimuth:d']

    X['snow_depth*temp*GHI'] = X['snow_depth:cm'] * X['temp*GHI']

    X['GHI_lag_interaction'] = X['GHI_lag-1'] * X['GHI_lag-2'] * X['GHI_lag1'] * X['GHI_lag2']
    X['GHI_lag_interaction_all'] = X['GHI_lag-1'] * X['GHI_lag-2'] * X['GHI_lag1'] * X['GHI_lag2'] * X['GHI']

    X['wind_speed*temp*GHI'] = X['wind_speed_10m:ms'] * X['t_1000hPa:K'] * X['GHI']

    X['cloud_base*clear_sky_energy'] = X['cloud_base_agl:m'] * X['clear_sky_energy_1h:J']

    X['precip*sun_elevation'] = X['precip_5min:mm'] * X['sun_elevation:d']
    X['supercooled_water*wind_speed'] = X['super_cooled_liquid_water:kgm2'] * X['wind_speed_10m:ms']

    return X




In [89]:
def add_features_mean(X):
    X['GHI_mean'] = X.groupby(['building_id', 'dayMonthYear'])['GHI'].transform('mean')

    return X

In [90]:
def add_features_differences(X):
    X['GHI_0_minus_-1'] = X['GHI'] - X['GHI_lag-1']
    X['GHI_0_minus_-2'] = X['GHI'] - X['GHI_lag-2']
    X['GHI_0_minus_1'] = X['GHI'] - X['GHI_lag1']
    X['GHI_0_minus_2'] = X['GHI'] - X['GHI_lag2']

    return X

In [91]:
# TODO: how to handle function parameters? have to set them for test and submission

def add_features(X):
    X = add_features_time(X)
    X = add_features_general(X)
    X = add_features_lag(X)
    X = add_features_interaction(X)
    X = add_features_mean(X)
    X = add_features_differences(X)
    return X


In [92]:
drop_cols = ['time', 'date_forecast', 'snow_density:kgm3', 'date_calc', 'monthYear', 'dayMonthYear']


In [93]:
"""
Optuna example that optimizes a classifier configuration for cancer dataset
using XGBoost.

In this example, we optimize the validation accuracy of cancer detection
using XGBoost. We optimize both the choice of booster model and its
hyperparameters.

"""

def objective(trial):
    # Load data
    X_train, y_train, _, _ = get_splitted_data()

    # Add features
    X_train = add_features(X_train)

    # shuffle data
    if trial.suggest_categorical('shuffle', [True, False]):
        X_train = X_train.sample(frac=1, random_state=42).reset_index(drop=True)

    # drop columns
    X_train = X_train.drop(columns=drop_cols,errors='ignore')

    categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
    impute_features = X_train.loc[:, X_train.isna().any()].columns.tolist()



    # set column transformer
    columnTransformer = ColumnTransformer(
        transformers=[
            ('imputer', SimpleImputer(strategy='constant'),impute_features),
            ('oneHotEncoder', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ],
        remainder='passthrough',  # Dont drop remaining columns
        n_jobs=-1
    )

    # build the pipeline
    pipeline = Pipeline(steps=[
        ('columnTransformer', columnTransformer),
        ('statusSaver', StatusSaver()),
        ('estimator', xgboost.XGBRegressor(
            random_state=42,
            learning_rate=0.1,
            max_depth=6,
            reg_alpha=8,
            reg_lambda=5,
            n_estimators=trial.suggest_int('n_estimators', 100, 1000, 100),
            colsample_bytree=1,
            min_child_weight=3,
            ))
    ])

    scores = cross_val_score(pipeline, X_train, y_train, cv=5)
    return scores.mean()

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=60)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))


[I 2023-11-02 02:29:02,497] A new study created in memory with name: no-name-296af20a-2474-47be-a907-ad638b9acfc1


/var/folders/s_/tyyjfphn3wl93jz8bhkrh3x40000gn/T/ipykernel_82483/3772693633.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['temp*GHI'] = X['GHI'] * X['t_1000hPa:K']
/var/folders/s_/tyyjfphn3wl93jz8bhkrh3x40000gn/T/ipykernel_82483/3772693633.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['wind*humidity'] = X['wind_speed_10m:ms'] * X['relative_humidity_1000hPa:p']
/var/folders/s_/tyyjfphn3wl93jz8bhkrh3x40000gn/T/ipykernel_82483/3772693633.py:5: SettingWithCopyWarning: 
A value is trying to be se

ValueError: Found input variables with inconsistent numbers of samples: [92450, 92462]

# Settings

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)


# Read data

In [ ]:
train_a = pd.read_parquet('A/train_targets.parquet')
train_b = pd.read_parquet('B/train_targets.parquet')
train_c = pd.read_parquet('C/train_targets.parquet')

X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')

X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')

X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('C/X_test_estimated.parquet')

# Adding building ID

In [ ]:
train_a['building_id'] = 'a'
train_b['building_id'] = 'b'
train_c['building_id'] = 'c'

X_train_estimated_a['building_id'] = 'a'
X_train_estimated_b['building_id'] = 'b'
X_train_estimated_c['building_id'] = 'c'

X_train_observed_a['building_id'] = 'a'
X_train_observed_b['building_id'] = 'b'
X_train_observed_c['building_id'] = 'c'

X_test_estimated_a['building_id'] = 'a'
X_test_estimated_b['building_id'] = 'b'
X_test_estimated_c['building_id'] = 'c'

# Combine data

In [ ]:
# Combine Data
X_o = pd.concat([X_train_observed_a, X_train_observed_b, X_train_observed_c])
X_e = pd.concat([X_train_estimated_a, X_train_estimated_b, X_train_estimated_c])
X_submission = pd.concat([X_test_estimated_a, X_test_estimated_b, X_test_estimated_c])
y = pd.concat([train_a, train_b, train_c])

# Add isEstimated column
X_o['isEstimated'] = 0
X_e['isEstimated'] = 1
X_submission['isEstimated'] = 1


# Combine
X = pd.concat([X_o, X_e])

# Add time column that only holds the hour
X['time'] = X['date_forecast'].dt.floor('H')
X_submission['time'] = X_submission['date_forecast'].dt.floor('H')

# combine X and y
Xy = pd.merge(X, y, on=['building_id', 'time'], how='inner')

# Add monthYear column
Xy['monthYear'] = Xy['date_forecast'].dt.to_period('M')
X_submission['monthYear'] = X_submission['date_forecast'].dt.to_period('M')

# Add dayMonthYear column
Xy['dayMonthYear'] = Xy['date_forecast'].dt.to_period('D')
X_submission['dayMonthYear'] = X_submission['date_forecast'].dt.to_period('D')

# Add month column
Xy['month'] = Xy['date_forecast'].dt.month
X_submission['month'] = X_submission['date_forecast'].dt.month

# Add hour column
Xy['hour'] = Xy['date_forecast'].dt.hour
X_submission['hour'] = X_submission['date_forecast'].dt.hour

# Prepare for joining ->The data is grouped per building and hour
Xy = Xy.groupby(['building_id', 'time']).mean().reset_index()
X_submission = X_submission.groupby(['building_id', 'time']).mean().reset_index()

# Create additional feature for estimated data "delta_forecast"
Xy['delta_forecast'] = (Xy['time']-Xy['date_calc']).apply(lambda x: x.total_seconds() / 3600)
X_submission['delta_forecast'] = (X_submission['time']-X_submission['date_calc']).apply(lambda x: x.total_seconds() / 3600)





# Preprocessing

# Feature engineering

In [ ]:
max_val = 1

Xy['GHI'] = Xy['diffuse_rad:W'] + Xy['direct_rad:W']
Xy['wind_angle'] = np.arctan2(Xy['wind_speed_u_10m:ms'], Xy['wind_speed_v_10m:ms'])
Xy['norm_radiation'] = (Xy['GHI'] / Xy['clear_sky_rad:W']).fillna(0)
Xy.loc[Xy['norm_radiation'] > max_val, 'norm_radiation'] = max_val

X_submission['GHI'] = X_submission['diffuse_rad:W'] + X_submission['direct_rad:W']
X_submission['wind_angle'] = np.arctan2(X_submission['wind_speed_u_10m:ms'], X_submission['wind_speed_v_10m:ms'])
X_submission['norm_radiation'] = (X_submission['GHI'] / X_submission['clear_sky_rad:W']).fillna(0)
X_submission.loc[X_submission['norm_radiation'] > max_val, 'norm_radiation'] = max_val


# Lagged Features

In [ ]:
# lagged feature of GHI
Xy['GHI_lag-2'] = Xy.groupby('building_id')['GHI'].shift(-2)
Xy['GHI_lag-1'] = Xy.groupby('building_id')['GHI'].shift(-1)
Xy['GHI_lag1'] = Xy.groupby('building_id')['GHI'].shift(1)
Xy['GHI_lag2'] = Xy.groupby('building_id')['GHI'].shift(2)

X_submission['GHI_lag-2'] = X_submission.groupby('building_id')['GHI'].shift(-2)
X_submission['GHI_lag-1'] = X_submission.groupby('building_id')['GHI'].shift(-1)
X_submission['GHI_lag1'] = X_submission.groupby('building_id')['GHI'].shift(1)
X_submission['GHI_lag2'] = X_submission.groupby('building_id')['GHI'].shift(2)

# remove rows were lagged features are nan
Xy = Xy.dropna(subset=['GHI_lag-2', 'GHI_lag-1', 'GHI_lag1', 'GHI_lag2'])


# Daily Mean

In [ ]:
# add daily mean of GHI
Xy['GHI_mean'] = Xy.groupby(['building_id', 'dayMonthYear'])['GHI'].transform('mean')
X_submission['GHI_mean'] = X_submission.groupby(['building_id', 'dayMonthYear'])['GHI'].transform('mean')



# Feature differences  

In [ ]:
Xy['GHI_0_minus_-1'] = Xy['GHI'] - Xy['GHI_lag-1']
Xy['GHI_0_minus_-2'] = Xy['GHI'] - Xy['GHI_lag-2']
Xy['GHI_0_minus_1'] = Xy['GHI'] - Xy['GHI_lag1']
Xy['GHI_0_minus_2'] = Xy['GHI'] - Xy['GHI_lag2']

X_submission['GHI_0_minus_-1'] = X_submission['GHI'] - X_submission['GHI_lag-1']
X_submission['GHI_0_minus_-2'] = X_submission['GHI'] - X_submission['GHI_lag-2']
X_submission['GHI_0_minus_1'] = X_submission['GHI'] - X_submission['GHI_lag1']
X_submission['GHI_0_minus_2'] = X_submission['GHI'] - X_submission['GHI_lag2']

# Feature Interactions

In [ ]:
Xy['temp*GHI'] = Xy['GHI'] * Xy['t_1000hPa:K']
Xy['wind*humidity'] = Xy['wind_speed_10m:ms'] * Xy['relative_humidity_1000hPa:p']
Xy['sun_height*diff_rad'] = Xy['sun_elevation:d'] * Xy['diffuse_rad:W']

# chat GPT
Xy['hour*wind_speed_10m'] = Xy['hour'] * Xy['wind_speed_10m:ms']
Xy['hour*clear_sky_rad'] = Xy['hour'] * Xy['clear_sky_rad:W']
Xy['month*sun_elevation'] = Xy['month'] * Xy['sun_elevation:d']

Xy['relative_humidity*air_density'] = Xy['relative_humidity_1000hPa:p'] * Xy['air_density_2m:kgm3']
Xy['temperature*wind_speed'] = Xy['t_1000hPa:K'] * Xy['wind_speed_10m:ms']

Xy['GHI*clear_sky_energy'] = Xy['GHI'] * Xy['clear_sky_energy_1h:J']
Xy['GHI*sun_azimuth'] = Xy['GHI'] * Xy['sun_azimuth:d']

Xy['snow_depth*temp*GHI'] = Xy['snow_depth:cm'] * Xy['temp*GHI']

Xy['GHI_lag_interaction'] = Xy['GHI_lag-1'] * Xy['GHI_lag-2'] * Xy['GHI_lag1'] * Xy['GHI_lag2']
Xy['GHI_lag_interaction_all'] = Xy['GHI_lag-1'] * Xy['GHI_lag-2'] * Xy['GHI_lag1'] * Xy['GHI_lag2'] * Xy['GHI']

Xy['wind_speed*temp*GHI'] = Xy['wind_speed_10m:ms'] * Xy['t_1000hPa:K'] * Xy['GHI']

Xy['cloud_base*clear_sky_energy'] = Xy['cloud_base_agl:m'] * Xy['clear_sky_energy_1h:J']

Xy['precip*sun_elevation'] = Xy['precip_5min:mm'] * Xy['sun_elevation:d']
Xy['supercooled_water*wind_speed'] = Xy['super_cooled_liquid_water:kgm2'] * Xy['wind_speed_10m:ms']


In [ ]:
X_submission['temp*GHI'] = X_submission['GHI'] * X_submission['t_1000hPa:K']
X_submission['wind*humidity'] = X_submission['wind_speed_10m:ms'] * X_submission['relative_humidity_1000hPa:p']
X_submission['sun_height*diff_rad'] = X_submission['sun_elevation:d'] * X_submission['diffuse_rad:W']

# chat GPT
X_submission['hour*wind_speed_10m'] = X_submission['hour'] * X_submission['wind_speed_10m:ms']
X_submission['hour*clear_sky_rad'] = X_submission['hour'] * X_submission['clear_sky_rad:W']
X_submission['month*sun_elevation'] = X_submission['month'] * X_submission['sun_elevation:d']

X_submission['relative_humidity*air_density'] = X_submission['relative_humidity_1000hPa:p'] * X_submission['air_density_2m:kgm3']
X_submission['temperature*wind_speed'] = X_submission['t_1000hPa:K'] * X_submission['wind_speed_10m:ms']

X_submission['GHI*clear_sky_energy'] = X_submission['GHI'] * X_submission['clear_sky_energy_1h:J']
X_submission['GHI*sun_azimuth'] = X_submission['GHI'] * X_submission['sun_azimuth:d']

X_submission['snow_depth*temp*GHI'] = X_submission['snow_depth:cm'] * X_submission['temp*GHI']

X_submission['GHI_lag_interaction'] = X_submission['GHI_lag-1'] * X_submission['GHI_lag-2'] * X_submission['GHI_lag1'] * X_submission['GHI_lag2']
X_submission['GHI_lag_interaction_all'] = X_submission['GHI_lag-1'] * X_submission['GHI_lag-2'] * X_submission['GHI_lag1'] * X_submission['GHI_lag2'] * X_submission['GHI']

X_submission['wind_speed*temp*GHI'] = X_submission['wind_speed_10m:ms'] * X_submission['t_1000hPa:K'] * X_submission['GHI']

X_submission['cloud_base*clear_sky_energy'] = X_submission['cloud_base_agl:m'] * X_submission['clear_sky_energy_1h:J']

X_submission['precip*sun_elevation'] = X_submission['precip_5min:mm'] * X_submission['sun_elevation:d']
X_submission['supercooled_water*wind_speed'] = X_submission['super_cooled_liquid_water:kgm2'] * X_submission['wind_speed_10m:ms']

# Preparing X and y

In [ ]:
Xy.columns

In [ ]:
# drop empty pv_measurement
Xy = Xy.dropna(subset=['pv_measurement'])

test_idx = Xy['date_forecast'].between('2021/05/01','2021/07/01') 

Xy_train = Xy[~test_idx].reset_index(drop=True)
Xy_test = Xy[test_idx].reset_index(drop=True)




In [ ]:
# Scale y
y_scaler = Y_Scaler_MaxAbs_per_building()
y_train = y_scaler.fit_transform(y_train, Xy_train['building_id'])
#y_o = y_scaler.fit_transform(y_o, X_o['building_id'])
#y_e = y_scaler.transform(y_e, X_e['building_id']) # no fit_transform because we use y_e as test data


# Scale whole y
# full_scaler = RobustScaler()
# y_o = full_scaler.fit_transform(y_o.values.reshape(-1, 1)).flatten()
# y_e = full_scaler.transform(y_e.values.reshape(-1, 1)).flatten() # no fit_transform because we use y_e as test data



In [ ]:
print(f"X.shape: {Xy_train.shape}")
print(f"X.shape: {Xy_test.shape}")


In [ ]:
# drop irrelevant columns
drop_cols = ['time', 'date_forecast', 'snow_density:kgm3',
             'date_calc', 'monthYear', 'dayMonthYear']

drop2 = ['clear_sky_energy_1h:J',
         'direct_rad_1h:J',
         'fresh_snow_24h:cm',
         'fresh_snow_1h:cm',
         'fresh_snow_12h:cm',
         'diffuse_rad_1h:J',
         'dew_point_2m:K',
         'dew_or_rime:idx',
         'precip_5min:mm',
         'fresh_snow_6h:cm',
         'prob_rime:p',
         'ceiling_height_agl:m',
         'rain_water:kgm2',
         'sfc_pressure:hPa',
         'snow_depth:cm',
         'snow_drift:idx',
         'snow_melt_10min:mm',
         'snow_water:kgm2',
         'pressure_50m:hPa',
         'wind_speed_w_1000hPa:ms',
         'pressure_100m:hPa',
         'fresh_snow_3h:cm']

drop_cols = drop_cols  # + drop2

print(drop_cols)
# ignore if column does not exist
X_train = X_train.drop(drop_cols, axis=1, errors='ignore')
X_test = X_test.drop(drop_cols, axis=1, errors='ignore')
# ignore if column does not exist
X = X.drop(drop_cols, axis=1, errors='ignore')
X_submission = X_submission.drop(drop_cols, axis=1, errors='ignore')


In [ ]:
# automatically set types of columns for imputing and oneHotEncoding
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
impute_features = X_train.loc[:, X.isna().any()].columns.tolist()

print(f"categorical_features: {categorical_features}")
print(f"impute_features: {impute_features}")


# Building the pipeline

In [ ]:
# create empty txt file to store status
open('status.csv', 'w').close()


In [ ]:


# BayesSearchCV
parameters_bayes = {
    'estimator__n_estimators': Integer(30,500),
    'estimator__max_depth': Integer(6, 14),
    'estimator__learning_rate': Real(0.01, 0.3),
    # 'estimator__subsample': Real(0.5, 1.0),
    # 'estimator__colsample_bytree': Real(0.8, 1.0),
    # 'estimator__colsample_bylevel': Real(0.8, 1.0),
    # 'estimator__colsample_bynode': Real(0.8, 1.0),
    #'estimator__gamma': Real(0, 2),
    'estimator__reg_alpha': Real(0, 10),
    'estimator__reg_lambda': Real(1, 10),
    'estimator__min_child_weight': Integer(1, 10),
    #'estimator__max_delta_step': Integer(0, 5)
}

parameters_grid = {
    'estimator__n_estimators': [20,25,30,35,40,45,50,55,60,65,70,75,100,150,200,250,300],
    # 'estimator__max_depth': list(range(3,14)),
    # 'estimator__learning_rate': [0.3]#np.arange(0.01, 0.2, 0.01)
    # 'estimator__subsample': [0.9,1],
    # 'estimator__colsample_bytree': np.arange(0.3, 1, 0.01),
    # 'estimator__gamma': [0, 0.5, 1, 1.5, 2],
    # 'estimator__reg_alpha': np.arange(0, 10, 0.5),
    # 'estimator__reg_lambda': np.arange(0, 10, 0.5),
    # 'estimator__min_child_weight': np.arange(0.5, 20, 0.5),
    # 'estimator__n_estimators': [75],

}


In [ ]:

# set column transformer
columnTransformer = ColumnTransformer(
    transformers=[
        ('imputer', SimpleImputer(strategy='constant'),impute_features),
        ('oneHotEncoder', OneHotEncoder(handle_unknown='ignore'), categorical_features),
    ],
    remainder='passthrough',  # Dont drop remaining columns
    n_jobs=-1
)

# build the pipeline
pipeline = Pipeline(steps=[
    ('columnTransformer', columnTransformer),
    ('statusSaver', StatusSaver()),
    ('estimator', xgboost.XGBRegressor(
        random_state=42,
        learning_rate=0.1,
        max_depth=6,
        reg_alpha=8,
        reg_lambda=5,
        n_estimators=45,
        colsample_bytree=1,
        min_child_weight=3,
        ))
])





# create bayesian search estimator
m1_BayesCV = BayesSearchCV(
    pipeline, parameters_bayes, scoring='neg_mean_absolute_error', cv=6, error_score='raise',n_points=6, n_jobs=-1, verbose=2, n_iter=1080, random_state=42)


m1_GridCV = GridSearchCV(
    pipeline, parameters_grid, scoring='neg_mean_absolute_error', cv=ps, error_score='raise', n_jobs=-1, verbose=2, refit=True)

# switch between BayesCV and GridCV
m1_CV = m1_GridCV

# fit the estimator on the data
#m1_CV.fit(X_o, y_o)
m1_CV.fit(X_train, y_train)

# get best model 
m1 = m1_CV.best_estimator_


In [ ]:
# print the scores
print('Best score:', m1_CV.best_score_)
print('Best parameters:', m1_CV.best_params_)


In [ ]:
#sns.lineplot(m1_CV.cv_results_['mean_test_score'])

In [ ]:

for key, values in m1_CV.param_grid.items():
    pass

ax = plt.gca()

sns.lineplot(y=m1_CV.cv_results_['mean_test_score'], x=list(map(str, values)))
plt.xticks(rotation=45);


In [ ]:
#m1.steps[-3][1].get_feature_names_out()


In [ ]:
f_names = m1.steps[-3][1].get_feature_names_out()
f_importances = m1.steps[-1][1].feature_importances_

f_importances_df = pd.DataFrame({'feature': f_names, 'importance': f_importances})

f_importances_df = f_importances_df.sort_values(by='importance', ascending=False)

f_importances_df.plot.bar(x='feature', y='importance', figsize=(20, 5))


In [ ]:
if type(m1_CV) == BayesSearchCV:
    _ = plot_objective(m1_CV.optimizer_results_[0])
    plt.show()

# Test model on test data

In [ ]:
# predict on estimated data
m1_pred = pd.Series(m1.predict(X_test))
t=m1_pred.copy()
#m1_pred = pd.Series(full_scaler.inverse_transform(m1_pred.values.reshape(-1, 1)).flatten())
m1_pred = y_scaler.inverse_transform(m1_pred, X_test['building_id'])
Xy_test['m1_pred'] = m1_pred

# calculate abs diff
Xy_test['abs_diff'] = np.abs(Xy_test['pv_measurement'] - Xy_test['m1_pred'])
Xy_test['diff'] = (Xy_test['pv_measurement'] - Xy_test['m1_pred'])

# calculate mae
mae = Xy_test['abs_diff'].mean()
print('MAE:', mae)


In [ ]:
sns.lineplot(data=Xy_test, x='time', y='pv_measurement', hue='building_id', legend=False)
plt.xticks(rotation=90);


In [ ]:
sns.lineplot(data=Xy_test, x='time', y='diff', hue='building_id', legend=False)
plt.xticks(rotation=90);


In [ ]:
# Creating the submission file
m1.fit(X, y)

# prepare dataframes
y_test_pred = pd.Series(m1.predict(X_submission))
# y_test_pred = pd.Series(full_scaler.inverse_transform(
#     y_test_pred.values.reshape(-1, 1)).flatten())
#y_test_pred = y_scaler.inverse_transform(y_test_pred, X_t['building_id']).copy()

# remove negative predictions
y_test_pred.iloc[y_test_pred < 0] = 0

# rename columns etc.
y_test_pred = y_test_pred.reset_index().rename(
    columns={'pv_measurement': 'prediction', 'index': 'id'})

# save submission file
y_test_pred.to_csv(
    'feature_extraction.csv', index=False, header=True)


#